# Endoscopy image augmentation

In [136]:
import pandas as pd
import numpy as np

import cv2
import matplotlib.pyplot as plt
import os

In [137]:
verbose = False

In [138]:
img_folder = r'C:\Users\user\ML\endoscopy\kvasir-capsule'

num_images_per_class = []
class_name = []

empty_images = []
empty_index = []

# if the dir has a metadata.csv file
if os.path.isfile(img_folder+'/metadata.csv'):
    df = pd.read_csv(img_folder+'/metadata.csv', sep=';')
    
df = df[['filename', 'video_id', 'frame_number', 'finding_category', 'finding_class', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4']]
df.columns = ['filename', 'video_id', 'frame_number', 'category', 'label', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4']
df



,filename,video_id,frame_number,category,label,x1,y1,x2,y2,x3,y3,x4,y4
0,0728084c8da942d9_22803.jpg,0728084c8da942d9,22803,Luminal,Normal clean mucosa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0728084c8da942d9_22804.jpg,0728084c8da942d9,22804,Luminal,Normal clean mucosa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0728084c8da942d9_22805.jpg,0728084c8da942d9,22805,Luminal,Normal clean mucosa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0728084c8da942d9_22806.jpg,0728084c8da942d9,22806,Luminal,Normal clean mucosa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0728084c8da942d9_22807.jpg,0728084c8da942d9,22807,Luminal,Normal clean mucosa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47243,04a78ef00c5245e0_898.jpg,04a78ef00c5245e0,898,Luminal,Angiectasia,171.0,172.0,237.0,172.0,237.0,238.0,171.0,238.0
47244,04a78ef00c5245e0_899.jpg,04a78ef00c5245e0,899,Luminal,Angiectasia,169.0,205.0,234.0,205.0,234.0,273.0,169.0,273.0
47245,04a78ef00c5245e0_900.jpg,04a78ef00c5245e0,900,Luminal,Angiectasia,174.0,240.0,242.0,240.0,242.0,305.0,174.0,305.0
47246,04a78ef00c5245e0_901.jpg,04a78ef00c5245e0,901,Luminal,Angiectasia,188.0,265.0,257.0,265.0,257.0,330.0,188.0,330.0


In [139]:
class_name = list(set(list(df['label'])))
for cn in class_name:
    count = 0
    for i in range(len(df)):
        if df.loc[i, 'label'] == cn:
            count += 1
    num_images_per_class.append(count)
    print(cn, count)

Foreign Body 776
Blood - fresh 446
Erosion 507
Angiectasia 866
Ampulla of Vater 10
Blood - hematin 12
Lymphangiectasia 592
Ulcer 854
Normal clean mucosa 34338
Pylorus 1538
Reduced Mucosal View 2906
Ileocecal valve 4189
Polyp 55
Erythema 159


In [140]:
videos_list = list(set(list(df['video_id'])))
print(videos_list)
print(len(videos_list))

df['video_id'].value_counts()

['0728084c8da942d9', 'dac1e27f7e4d4ef5', '89cdd41258c542c5', '6cb700585c4f4070', '8885668afb844852', '07c1fa15a20a4398', 'd626f4f4a5ac4785', 'af9bd7d0e43741e3', 'ed02f27ef36f483e', '0531325b64674948', 'ad91cf7ca91440aa', 'bca26705313a4644', '5bb1d3cc7dc64cec', 'c7084b3556e34619', 'dca1377dec974312', '7a47e8eacea04e64', '7ad22d50ebaf4596', '8ebf0e483cac48d6', '5e9beaf4e66142c8', 'd7a271f233ba4a40', '495f16498db34d3c', 'df6b47bafe5143f5', 'fc32def0e7194981', '8a00709108cd4e2b', '5e59c7fdb16c4228', '48579eec79784294', '3ada4222967f421d', '3c8d5f0b90d7475d', 'dc221ccc65d34010', '131368cc17e44240', '2fc3db471f9d44c0', '4aebc5cb2d4847aa', 'b2134f4a6f864613', '4560e83f9afc4685', 'fb86bc87d3874cd7', '39960e5e099a45ca', 'bc84479c66fe4da6', 'eb0203196e284797', 'c11b28a8b2344716', 'fe5d372e43f94f68', '64440803f87b4843', 'd369e4f163df4aba', '04a78ef00c5245e0']
43


video_id
5e59c7fdb16c4228    2394
64440803f87b4843    2159
dac1e27f7e4d4ef5    2152
fb86bc87d3874cd7    2086
131368cc17e44240    2034
8885668afb844852    2021
bc84479c66fe4da6    1891
dc221ccc65d34010    1789
3c8d5f0b90d7475d    1704
df6b47bafe5143f5    1530
0728084c8da942d9    1499
bca26705313a4644    1449
fc32def0e7194981    1380
48579eec79784294    1379
d369e4f163df4aba    1328
b2134f4a6f864613    1297
04a78ef00c5245e0    1292
dca1377dec974312    1216
89cdd41258c542c5    1198
0531325b64674948    1191
8ebf0e483cac48d6    1077
3ada4222967f421d    1002
c11b28a8b2344716    1000
c7084b3556e34619     976
d7a271f233ba4a40     945
6cb700585c4f4070     884
eb0203196e284797     881
5bb1d3cc7dc64cec     788
4560e83f9afc4685     780
7ad22d50ebaf4596     740
07c1fa15a20a4398     678
d626f4f4a5ac4785     643
ed02f27ef36f483e     587
fe5d372e43f94f68     554
8a00709108cd4e2b     542
5e9beaf4e66142c8     464
7a47e8eacea04e64     436
af9bd7d0e43741e3     431
495f16498db34d3c     323
39960e5e099a45ca

In [141]:
class_of_interest = 'Erythema'

df_class_of_interest = df[df['label'] == class_of_interest]
df_class_of_interest = df_class_of_interest.reset_index(drop=True)

df_class_of_interest

,filename,video_id,frame_number,category,label,x1,y1,x2,y2,x3,y3,x4,y4
0,5e59c7fdb16c4228_37944.jpg,5e59c7fdb16c4228,37944,Luminal,Erythema,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5e59c7fdb16c4228_37945.jpg,5e59c7fdb16c4228,37945,Luminal,Erythema,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5e59c7fdb16c4228_37946.jpg,5e59c7fdb16c4228,37946,Luminal,Erythema,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5e59c7fdb16c4228_36836.jpg,5e59c7fdb16c4228,36836,Luminal,Erythema,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5e59c7fdb16c4228_36837.jpg,5e59c7fdb16c4228,36837,Luminal,Erythema,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,48579eec79784294_29037.jpg,48579eec79784294,29037,Luminal,Erythema,55.0,241.0,187.0,241.0,187.0,323.0,55.0,323.0
155,48579eec79784294_29038.jpg,48579eec79784294,29038,Luminal,Erythema,77.0,239.0,210.0,239.0,210.0,322.0,77.0,322.0
156,48579eec79784294_29039.jpg,48579eec79784294,29039,Luminal,Erythema,104.0,238.0,223.0,238.0,223.0,322.0,104.0,322.0
157,48579eec79784294_29040.jpg,48579eec79784294,29040,Luminal,Erythema,104.0,238.0,230.0,238.0,230.0,318.0,104.0,318.0


In [142]:
bbox = list(df_class_of_interest.iloc[0, 5:])
bbox

[nan, nan, nan, nan, nan, nan, nan, nan]

In [143]:
def rotate_image(image, angle, bbox):


    h, w = img.shape[:2]
    cX, cY = (w // 2, h // 2)

    # get rotation matrix (explained in section below)
    M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)

    # get cos and sin value from the rotation matrix
    cos, sin = abs(M[0, 0]), abs(M[0, 1])

    # calculate new width and height after rotation (explained in section below)
    newW = int((h * sin) + (w * cos))
    newH = int((h * cos) + (w * sin))

    # calculate new rotation center
    M[0, 2] += (newW / 2) - cX
    M[1, 2] += (newH / 2) - cY

    # use modified rotation center and rotation matrix in the warpAffine method
    img_rotated = cv2.warpAffine(img, M, (newW, newH)) 


    # h, w = img.shape[:2] 
    # cx, cy = (int(w / 2), int(h / 2))

    bbox_tuple = [
            (bbox[0], bbox[1]),
            (bbox[2], bbox[3]),
            (bbox[4], bbox[5]),
            (bbox[6], bbox[7]),
        ] # put x and y coordinates in tuples, we will iterate through the tuples and perform rotation

    if bbox[0] == np.NaN:
        bbox_rotated = [np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN]
    else:
        rotated_bbox = []

        for i, coord in enumerate(bbox_tuple):

            M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)
            cos, sin = abs(M[0, 0]), abs(M[0, 1])
            newW = int((h * sin) + (w * cos))
            newH = int((h * cos) + (w * sin))
            M[0, 2] += (newW / 2) - cX
            M[1, 2] += (newH / 2) - cY
            v = [coord[0], coord[1], 1]
            adjusted_coord = np.dot(M, v)
            rotated_bbox.insert(i, (adjusted_coord[0], adjusted_coord[1]))

        bbox_rotated = [int(x) for t in rotated_bbox for x in t]

        #print(bbox_rotated)

        #xmin, ymin, xmax, ymax = result[0], result[1], result[4], result[5]
        #cv2.rectangle(result, (xmin, ymin), (xmax, ymax), (0, 0, 255), thickness=2, lineType=cv2.LINE_AA)

        #plt.figure(figsize=[5,5])
        #plt.imshow(img_rotated[:,:,::-1])

    return img_rotated, bbox_rotated

In [144]:
def nbi_image(img):
    """transforms an RGB image into a pseudo-narrow band image"""

    nbi = img.copy()
    if verbose == True:
        plt.imshow(nbi)
        plt.show()

    print(nbi.shape)

    # remove the red channel
    nbi[:,:,2] = np.zeros([nbi.shape[0], nbi.shape[1]])

    max_blue = np.max(nbi[:,:,0])

    # accentuate the blue channel
    for i in range(nbi.shape[0]):
        for j in range(nbi.shape[1]):

            # add blue light
            if nbi[i,j,0] < 245:
                nbi[i, j, 0] += 10
                if nbi[i, j, 0] > 255:
                    nbi[i, j, 0] = 255

    if verbose == True:
        plt.imshow(nbi)
        plt.show()

    return nbi



In [145]:
SIZE = 229

train_images = []
train_labels = []

rotations = [90, 180, 270]

verbose = False

for i in range(len(df_class_of_interest)):

    img_path = img_folder+'\\'+class_of_interest+'\\'+df_class_of_interest.loc[i, 'filename']
    #print(i, img_path)

    img = cv2.imread(img_path, cv2.IMREAD_COLOR)   
    #print(img.shape)
    bbox = list(df_class_of_interest.iloc[i, 5:])

    #print("error - did not read properly")    
    

    width = img.shape[0]
    height = img.shape[1]

    #img = cv2.resize(img, (SIZE, SIZE))
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    
    train_images.append(img)


    img_nbi = nbi_image(img)
    train_images.append(img_nbi)

    df_class_of_interest = pd.concat([df_class_of_interest, pd.DataFrame({'filename': df_class_of_interest.loc[i, 'filename'][:-4]+'_nbi.jpg',
                                                                                'video_id': df_class_of_interest.loc[i, 'video_id'],
                                                                                'frame_number': df_class_of_interest.loc[i, 'frame_number'],
                                                                                'category': df_class_of_interest.loc[i, 'category'],
                                                                                'label': df_class_of_interest.loc[i, 'label'],
                                                                                'x1': bbox[0],
                                                                                'y1': bbox[1],
                                                                                'x2': bbox[2],
                                                                                'y2': bbox[3],
                                                                                'x3': bbox[4],
                                                                                'y3': bbox[5],
                                                                                'x4': bbox[6],
                                                                                'y4': bbox[7]}, index=[len(df_class_of_interest)])], ignore_index=True)


    for r in rotations:

        img_rotated, bbox_rotated = rotate_image(img, r, bbox)

        #print(img_rotated.shape, bbox_rotated)
        train_images.append(img_rotated)

        df_class_of_interest = pd.concat([df_class_of_interest, pd.DataFrame({'filename': df_class_of_interest.loc[i, 'filename'][:-4]+'_rot'+str(r)+'.jpg',
                                                                                'video_id': df_class_of_interest.loc[i, 'video_id'],
                                                                                'frame_number': df_class_of_interest.loc[i, 'frame_number'],
                                                                                'category': df_class_of_interest.loc[i, 'category'],
                                                                                'label': df_class_of_interest.loc[i, 'label'],
                                                                                'x1': bbox_rotated[0],
                                                                                'y1': bbox_rotated[1],
                                                                                'x2': bbox_rotated[2],
                                                                                'y2': bbox_rotated[3],
                                                                                'x3': bbox_rotated[4],
                                                                                'y3': bbox_rotated[5],
                                                                                'x4': bbox_rotated[6],
                                                                                'y4': bbox_rotated[7]}, index=[len(df_class_of_interest)])], ignore_index=True)



                                
    #train_images.append(rotate_image(img, 180))
    #train_images.append(rotate_image(img, 270))


# for i in range(len(train_images)):
#     print(i)
#     plt.imshow(train_images[i])
#     plt.show()
    

    #r_img = np.reshape(img, (SIZE*SIZE*3))
    #train_images.append(r_img)
    #train_labels.append(y_train[i])

# train_images = np.array(train_images)
# train_labels = np.array(train_labels)
# train_images.shape

(336, 336, 3)


ValueError: cannot convert float NaN to integer

In [ ]:
df_class_of_interest

In [ ]:
verbose = True

nbi = nbi_image(train_images[0])